In [36]:
import simpy
import random

In [37]:
import simpy
import random

class ContainerTerminal:
    def __init__(self, env, num_berths, num_cranes, num_trucks):
        self.env = env
        self.berths = simpy.Resource(env, num_berths)
        self.cranes = simpy.Resource(env, num_cranes)
        self.trucks = simpy.Resource(env, num_trucks)
        self.truck_queue = simpy.Store(env, capacity=num_trucks)
        self.truck_list = [f"Truck_{i+1}" for i in range(num_trucks)]
        for truck in self.truck_list:
            self.truck_queue.put(truck)
        self.total_vessels = 0
        self.total_containers = 0

    def discharge_containers(self, vessel, crane_id, num_containers):
        for i in range(num_containers):
            print(f"Time {self.env.now}: Crane {crane_id} starts lifting container {i+1} from {vessel.name}")
            yield self.env.timeout(3)  # 3 minutes to move one container
            print(f"Time {self.env.now}: Crane {crane_id} finished lifting container {i+1} from {vessel.name}")
            
            truck = yield self.truck_queue.get()
            print(f"Time {self.env.now}: {truck} starts transporting container {i+1} from {vessel.name}")
            self.env.process(self.transport_container(truck))
            self.total_containers += 1

    def transport_container(self, truck):
        yield self.env.timeout(6)  # 6 minutes to transport  and return
        print(f"Time {self.env.now}: {truck} finished transporting container and is available again")
        yield self.truck_queue.put(truck)


In [38]:

class Vessel:
    def __init__(self, env, name, terminal):
        self.env = env
        self.name = name
        self.terminal = terminal
        self.action = env.process(self.arrive())

    def arrive(self):
        print(f"Time {self.env.now}: Vessel {self.name} arriving")
        with self.terminal.berths.request() as berth_request:
            yield berth_request
            berth_id = berth_request.resource.users.index(berth_request)
            print(f"Time {self.env.now}: Vessel {self.name} berthed at Berth_{berth_id + 1}")
            with self.terminal.cranes.request() as crane_request:
                yield crane_request
                crane_id = crane_request.resource.users.index(crane_request)
                print(f"Time {self.env.now}: Crane {crane_id + 1} assigned to {self.name}")
                yield self.env.process(self.terminal.discharge_containers(self, crane_id + 1, 150))
                print(f"Time {self.env.now}: Vessel {self.name} finished discharging and leaving Berth_{berth_id + 1}")
                self.terminal.total_vessels += 1

In [39]:
def vessel_arrival(env, terminal):
    vessel_id = 0
    while True:
        inter_arrival_time = random.expovariate(1/5)  # average 5 hours between arrivals
        yield env.timeout(inter_arrival_time * 60)  # converting hours to minutes
        vessel_id += 1
        Vessel(env, f"Vessel_{vessel_id}", terminal)


In [43]:
def main():
    SIMULATION_TIME = 34 * 60  #manipulatte and see magic
    random.seed(42)

    env = simpy.Environment()
    terminal = ContainerTerminal(env, num_berths=2, num_cranes=2, num_trucks=3)

    env.process(vessel_arrival(env, terminal))
    env.run(until=SIMULATION_TIME)

    # Summary
    print("\n" + "#" * 40)
    print(f"Simulation completed after {SIMULATION_TIME} minutes")
    print(f"Total vessels arrived and completed unloading: {terminal.total_vessels}")
    print(f"Total containers shifted: {terminal.total_containers}")
    print("#" * 40 + "\n")

if __name__ == "__main__":
    main()


Time 306.01808618244024: Vessel Vessel_1 arriving
Time 306.01808618244024: Vessel Vessel_1 berthed at Berth_1
Time 306.01808618244024: Crane 1 assigned to Vessel_1
Time 306.01808618244024: Crane 1 starts lifting container 1 from Vessel_1
Time 309.01808618244024: Crane 1 finished lifting container 1 from Vessel_1
Time 309.01808618244024: Truck_1 starts transporting container 1 from Vessel_1
Time 309.01808618244024: Crane 1 starts lifting container 2 from Vessel_1
Time 312.01808618244024: Crane 1 finished lifting container 2 from Vessel_1
Time 312.01808618244024: Truck_2 starts transporting container 2 from Vessel_1
Time 312.01808618244024: Crane 1 starts lifting container 3 from Vessel_1
Time 313.6167378952619: Vessel Vessel_2 arriving
Time 313.6167378952619: Vessel Vessel_2 berthed at Berth_2
Time 313.6167378952619: Crane 2 assigned to Vessel_2
Time 313.6167378952619: Crane 2 starts lifting container 1 from Vessel_2
Time 315.01808618244024: Truck_1 finished transporting container and i